# Tensorboard support

In [1]:
%pip install tensorboard matplotlib

  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached markdown-3.9-py3-none-any.whl.metadata (5.1 kB)
  Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached tensorboard-2.20.0-py3-none-any.whl (5.5 MB)
Using cached tensorboard_data_server-0.7.2-py3-none-any.whl (2.4 kB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 533.8 kB/s eta 0:00:0000:0100:01
Using cached markdown-3.9-py3-none-any.whl (107 kB)
Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl (426 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [tensorboard] [tensorboard]

[notice] A new release of pip is available

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from torch.utils.tensorboard import SummaryWriter

## Showing images in Tensorboard

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))]
)

# Store separate training nd validation splits in ./data
training_set = torchvision.datasets.FashionMNIST('./data',download=True, train=True, transform=transform)

validation_set = torchvision.datasets.FashionMNIST('./data',
    download=True, train=False, transform=transform)

training_loader = torch.utils.data.DataLoader(
    training_set, batch_size=4, shuffle=True, num_workers=2
)

validation_loader = torch.utils.data.DataLoader(
    validation_set, batch_size=4, shuffle=True, num_workers=2
)

# Class Labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Helper function for inline image display
def matplotlib_imshow(img, one_channels=False):
    if one_channel :
        img = img.mean(dim=0)
    img = img/2 + 0.5
    nping = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap='Greys')
    else:
        plt.imshow(np.transpose(npimg, (1,2,0)))

# Extract a batch of 4 images
dataiter = iter(training_laoder)
images, labels = next(dataiter)

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

In [ ]:
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

writer.add_image('Four Fashion_MNIST Images', img_grid)
writer.flush()

In [ ]:
!tensorboard --logdir=runs

## Graphing Scalers to Visualize Training

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Now let's train a single epoch and evaluate the training vs validation set losses every 1000 batches

In [ ]:
print(len(validation_loader))

for epoch in range(1):      # loop over the dataset multiple times
    running_loss = 0.0

    for i, data in enumerate(training_loader, 0):
        # basic training loop
        inputs, labels = data
        optimizer.zero_grad()
        output = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999 :        # every 1000 mini-batches ...
            print('Batch {}'.format(i+1))
            # check against the validation set
            running_vloss = 0.0

            # In evaluation mode, some model specific operations can be omitted eg. dropout layer
            net.train(False)    # switching to evaluation mode, eg. turning off regularisation
            for j, vdata in enumerate(validation_loader, 0):
                vinputs, vlabels = vdata
                voutputs = net(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
            net.train(True)     # switching back to training mode, eg. turning on regularisation
            avg_loss = running_loss / 1000
            avg_vloss = running_vloss / len(validation_loader)

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs Validation Loss', {'Training': avg_loss, 'Validation: ', avg_vloss}, epoch*len(training_loader) + i)
            running_loss = 0.0

print('Finished Training')
writer.flush()

## Visualizing your Model

TensorBoard can also be used to examine the data flow within your model. To do this, call the add_graph() method with a model and sample input.

When you switch over to TensorBoard, you should see a GRAPHS tab. Double-click the “NET” node to see the layers and data flow within your model.



In [ ]:
# grab a single mini batch
dataiter = iter(training_loader)
images, labels = next(dataiter)

# add_graph will trace the sample input through your model and render it as a graph
writer.add_graph(net, images)
writer.flush()

## Visualize your Dataset with Embeddings

The `add_embedding()` method will project a set of data onto the three dimensions with highest variance, and display them as an interactive 3D chart. The add_embedding() method does this automatically by projecting to the three dimensions with highest variance.

In [ ]:
# Select a random subset of data and corresponding labels
def select_n_random(data, labels, n=100):
    assert len(data) == len(labels)
    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# Extract a random subsets of data
images, labels = select_n_random(training_set.data, training_set.targets)

# get the class labels for each image
class_labels = [classes[label] for label in labels]

features = images.view(-1, 28*28)
writer.add_embedding(features,
    metadat=class_labels, label_img=images.unsqueeze(1))

writer.flush()
writer.close()